# Mapping Economic Development of Mini-grid Communities 

## Introduction

We'll use Python, Juypter notebooks and GitHub to create our insights. Each week we'll make additions to our code and to investigate this topic. 
This notebook is going to be a summary of all the data we have on Odyssey and what, how and why we'll use it to generate insights. 

## CBDA Rules 

- No creditentials to be stored on GitHub - API Secrets etc. should be stored in the user_config.yml which is ignored by git. 
- No Developer data is to be stored on GitHub 
- This data is confidential and any findings should be aggregated and anonymous 
- Before pushing to GitHub clear all outputs from a juypter notebook 



In [ ]:

import elasticsearch # required for connection
from elasticsearch import Elasticsearch, client
import yaml # for pulling credentials

# This is a very useful bit of code and we use it create HTML documents from Jupyter notebooks. 
# Compile using: "jupyter nbconvert introduction.ipynb --no-input --to html" to hide the code inputs 
import pandas as pd
import numpy as np
import plotly.express as px
import plotly
import os
from datetime import date
from datetime import datetime
import pytz

# We also use a repo_utils to store useful code throughout a project 
import repo_utils

plotly.offline.init_notebook_mode() # Allows publishing of notebook to HTML with plots. 

In [ ]:
# Establish connection to Odyssey
secrets_file_path = 'odyssey_secrets.yml'
with open(secrets_file_path, 'r') as file:
    secrets = yaml.safe_load(file)
id = secrets['api-id']
key = secrets['api-key']

host = "https://f05fc8c52e6b4a99b838818f59bbda80.us-east-1.aws.found.io:9243"
es = Elasticsearch([host], api_key=(id,key),request_timeout=60, max_retries=2, retry_on_timeout=True)
es.ping()
# str(type(es))
# output should be "true"

In [ ]:
# This code queries our Odyssey database, finding the lastest recording from each of the sites we have data for. 
# We use this to assess which sites have sent us data lately. 
query = """
        SELECT dataKey.project.id, dataKey.project.name,
            dataKey.organization.name, metadata.project.country, metadata.project.latitude as latitude, metadata.project.longitude as longitude, metadata.project.status,
            metadata.dataProvider, MAX(timestamp) as timestamp, COUNT(DISTINCT dataKey.meter.id) meter_count
            FROM daily_meter_summary_logs
            GROUP BY dataKey.project.id, dataKey.project.name,
            dataKey.organization.name, metadata.project.country, metadata.project.status,
            metadata.dataProvider, latitude, longitude
        """

sm_res = repo_utils.query(query, show_progress=True)

df = sm_res
# Get the most recent timestamp for each project.id. There are repeated values due to different data sources
df = df.sort_values(by=["site_id_odyssey", "timestamp"], ascending=False)
df = df.drop_duplicates(subset=["site_id_odyssey"], keep="first")

# Clean up the index
df = df.sort_values(by="timestamp", ascending=True)
df = df.reset_index(drop=True)

df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True)
df["elapsed_time"] = (datetime.now(tz=pytz.utc) - df["timestamp"]).dt.days
df["data_source"] = df["data_source"].str.replace("OES", "File upload", regex=True)
# df.drop(columns =["site", "metadata.project.status"])

df.to_csv("./data/daily_meter_summary_logs.csv", index=False)

Creating a simple visualisation to show the range of data freshness that we have and the size of the sites 

In [ ]:
fig = px.scatter(df, x = "timestamp", y = "meter_count", color = "country")
fig.show()

In answer to your questions: 

In [ ]:
print("We currently have ", len(df), "sites reporting data on our Odyssey database") # This gives you an idea of what sites we have and when they were last published.
print("Of these, there are: ", len(df[df["elapsed_time"] < 365 ]), " that have sent data to Odyssey in the last year.")
print("The latest total meter count is", sum(df["meter_count"]))
print("We have co-ordinates for: ", len(df[(df["elapsed_time"] < 365) & (~df["latitude"].isna()) & (~df["longitude"].isna()) ]), "sites")

Here is a sample of the revenue and consumption data we have. The consumption is in kWh, revenue is in local currency and the recordings are for one day across all of our sites showing individual meter recordings. 

In [ ]:
# Here is a sample of daily data we have from a day at random. 
query = """
        SELECT timestamp, SUM(meter.energyConsumptionKwh) consump_kwh, SUM(payment.amount) revenue_lc, dataKey.project.name site, dataKey.meter.id meter_id
        FROM daily_meter_summary_logs
        WHERE timestamp > '2021-05-30' AND timestamp < '2021-06-01' 
        GROUP BY timestamp, site, meter_id
"""

consumption_and_revenue_data = repo_utils.sql_to_df(query, show_progress=True)
consumption_and_revenue_data.drop(columns =["site"])

In [ ]:
fig = px.line(consumption_and_revenue_data, x = "timestamp", y = "consump_kwh", color = "meter_id")
fig.show()

In [ ]:
site_name = "SOLAR HYBRID MINI-GRID IN RUKUBI DOMA NASARAWA Final Approved"
site_name = repo_utils.parse_to_sql(site_name)

query = f"""
        SELECT dataKey.project.id, dataKey.project.name site,
            dataKey.organization.name, metadata.project.country, 
            metadata.project.latitude as latitude, metadata.project.longitude as longitude, metadata.project.status,
            metadata.dataProvider, MAX(timestamp) as timestamp, dataKey.meter.id meter_id
            
            FROM daily_meter_summary_logs
            
            GROUP BY dataKey.project.id, site, meter_id,
            dataKey.organization.name, metadata.project.country, metadata.project.status,
            metadata.dataProvider, latitude, longitude
        """

site_data = repo_utils.query(query, show_progress=True)

In [ ]:
site_data

# Actions 
- Explain the Odyssey eco-system (Good preparation for a handover of this work)
- Outline next steps for Isaiah to start investigating this data - Lets try and create just one plot a week to discuss. This will help us build a gallery of our research and makes it easier to create small LinkedIn posts that we can continuously share.  